# Download and process some Nextmap tiles from the CEDA archive


**Ensure you are running the correct kernel. Look at the top right  it should be [conda env:eot] as seen in the top right.** 

**If not, Kernel > Change kernel > eot from the menu.** 

The example is running on countryside survey data which may be accessed via the CEH environmental data centre. 

https://eip.ceh.ac.uk/

The data is returned as a dataframe as well as written to file and can be easily plotted using pandas native functions. 

If you wish to see how this has been done, please look at the files in src.

In [ ]:
from src.downloader import  dtmftp_mt
from src.utils import batch_translate_adf, batch_gdaldem, replace_str, zonal_point
import geopandas as gpd
import os
from shutil import rmtree
from glob2 import glob

### We need a load of OS grid tiles that contain the CSS survey plots.

This is most easily done with QGIS via Processing > Vector selection > Extract by location.

I used the a public 10km OSGB grid dataset to achieve this which is in this repo 
(osgb10kmgrid.shp)

A polygon from that operation will be used here to provide the names for the download.

**Please generate your own as the CSS poly can't be uploaded here.**

In [ ]:
inShp = 'path/to/CSS/shp'

gdf = gpd.read_file(inShp)

First we extract the tile names from the gpd

In [ ]:
tilenms = gdf["TILE_NAME"].tolist()

tilenms[0:3]

### Input/output admin

Some messing around is needed first to input the correct info into the download function. I have just copied the url from one dataset from ceda website and will alter it quickly here to create inputs to bulk download it


In [ ]:
template = "ftp://ftp.ceda.ac.uk/neodc/nextmap/by_tile/hp/hp40/dtm/hp40dtm/"

#the tile ids must be lower case
tilenms = [t.lower() for t in tilenms]

# insert the tileid for every item
dwnurls = [replace_str(template, t) for t in tilenms]

# the url has to have the ftp part removed
ftplist = [d.replace("ftp://ftp.ceda.ac.uk/", "") for d in dwnurls]

# example
ftplist[0:3]

### Download

Unfortunately CEDA's DAP server is not currently working for this dataset, meaning we have to resort to rather arcane FTP methods. The function below downloads data in parallel using ftplib.

Enter you user and password below to use this as with previous notebooks.

In [ ]:
user = ""
passwd =""

# download to here unless specified
main_dir = os.getcwd()

**This will take a while! Best to go do something!**

In [ ]:
outpaths = dtmftp_mt(user, passwd, ftplist, main_dir)

### Translate & tidy up

Unfortunately this data arrives in ESRI binary grid format, so best to convert it to something better.

In [ ]:
# list the binary grid dirs
dirlist = glob(os.path.join(os.getcwd(), '*dtm'))
# add the header
inlist = [os.path.join(d, 'hdr.adf') for d in dirlist]

Translate.....

In [ ]:
dtms = batch_translate_adf(inlist)

Remove ESRI garbage

In [ ]:
_ = [rmtree(d) for d in dirlist]

**Build a virtual raster of the DTMs (A ```!``` denotes cmd line use)**

In [ ]:
!gdalbuildvrt cssDTM.vrt *.tif

### Now calculate aspect and slope for each dtm

It would of course be inefficient mosaic then calculate aspect etc so we loop through them all and build virtual rasters at the end.


In [ ]:
aspects = batch_gdaldem(dtms, prop='aspect')

slopes = batch_gdaldem(dtms, prop='slope')

### Construct virtual rasters of each using the gdal command line


In [ ]:
!gdalbuildvrt cssAspect.vrt *aspect.tif

!gdalbuildvrt cssSlope.vrt *slope.tif


### Attribute CSS points

Now all that is required is a zonal point for each CSS plot.

Here the parameters are (in order) the input point file, input raster and the field name that will record the value.

In [ ]:
zonal_point(inShp, 'cssAspect.vrt', 'dtm-aspect')

zonal_point(inShp, 'cssSlope.vrt', 'dtm-slope')

zonal_point(inShp, 'cssDTM.vrt', 'dtm-elev')